In [30]:
from pyspark.sql import SparkSession, functions as F
from delta import configure_spark_with_delta_pip
import os

In [2]:
builder = SparkSession.builder \
    .appName("Delta Lake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/22 20:58:44 WARN Utils: Your hostname, provira-ERAZER-P6705-MD61203, resolves to a loopback address: 127.0.1.1; using 192.168.1.55 instead (on interface wlo1)
25/06/22 20:58:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/provira/anaconda3/envs/py_tfm_env/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/provira/.ivy2.5.2/cache
The jars for the packages stored in: /home/provira/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-540aa8f8-3012-40d5-a27f-026302049d33;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 247ms :: art

In [22]:
path_data = "./../../../datasets/"
types = ['csv', 'parquet', 'json']
sources = ['Kaggle', 'uci', 'aws']

In [ ]:
dfs = {}

for source in sources:
    for file_type in types:

        path = f'{path_data}{source}/{file_type}'
        if os.path.exists(path):
            
            df = spark.read.option("header", "true") \
                .option("recursiveFileLookup", "true") \
                .format(file_type) \
                .load(path)
            dfs[file_type] = df
            print(f"Loaded {df.count()} rows for type {file_type}")

Loaded 1100992 rows for type csv
Loaded 416809 rows for type parquet


In [37]:
for name, df in dfs.items():
    df.write.format("delta").mode("overwrite").save(f"./../../../delta_lake/{name}")

25/06/22 21:45:26 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 37, schema size: 3
CSV file: file:///home/provira/Documents/TFM/TFM/datasets/Kaggle/csv/GoEmotions/goemotions_1.csv
25/06/22 21:45:26 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 37, schema size: 3
CSV file: file:///home/provira/Documents/TFM/TFM/datasets/Kaggle/csv/GoEmotions/goemotions_3.csv
25/06/22 21:45:26 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 37, schema size: 3
CSV file: file:///home/provira/Documents/TFM/TFM/datasets/Kaggle/csv/GoEmotions/goemotions_2.csv
25/06/22 21:45:26 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 2, schema size: 3
CSV file: file:///home/provira/Documents/TFM/TFM/datasets/Kaggle/csv/DailyDialog.csv
25/06